In [4]:
import numpy as np
from matplotlib import pyplot as plt

from layers import Function, Layer, NeuralNetwork

import tensorflow_datasets as tfds
import tensorflow as tf

## Build dataset

In [35]:
BATCH_SIZE = m = 20

ds = tfds.load('mnist', split='train', shuffle_files=True)

# Build your input pipeline
ds = ds.shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

ds_val = tfds.load('mnist', split='test', shuffle_files=True)

# Build your input pipeline
ds_val = ds_val.shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

## Define NN

In [36]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    # one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y = np.zeros((Y.size, 10))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [37]:
def preprocess(example):
    image, label = example["image"], example["label"]
    
    # preprocessing
    image = np.resize(image, (BATCH_SIZE, 28*28))
    image = np.moveaxis(image, [0,1], [1,0])
    
    # label_vec = np.zeros((10, BATCH_SIZE))
    # for u, l in enumerate(label):
    #     label_vec[l, u] = 1

    # correct datatypes
    image = image.astype(np.float64)
    
    image /= 255.0
    
    return image, label

In [43]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions)
    print(Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i, elem in enumerate(tfds.as_numpy(ds)):
        X, Y = preprocess(elem)
        
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [44]:
W1, b1, W2, b2 = gradient_descent(0.10, 500)   

Iteration:  0
[5 6 0 3 5 5 1 0 0 0 5 5 0 0 0 5 5 3 0 3]
[7 3 5 8 6 5 7 3 3 3 8 7 8 3 8 9 6 9 6 5]
0.05
Iteration:  10
[2 6 8 6 0 5 3 1 6 9 0 1 7 5 7 2 2 2 1 8]
[5 0 1 2 3 8 2 8 2 9 2 4 1 4 7 5 3 7 7 1]
0.1
Iteration:  20
[9 1 8 9 7 8 0 8 2 1 6 2 2 1 1 1 6 3 5 8]
[3 1 2 7 4 9 3 3 6 2 5 1 1 7 1 3 6 4 0 4]
0.15
Iteration:  30
[4 8 3 4 7 1 1 2 9 1 1 0 3 2 1 0 3 1 4 9]
[9 9 4 4 9 2 7 3 9 1 7 0 3 2 0 0 5 1 4 4]
0.45
Iteration:  40
[9 7 0 0 0 0 4 8 6 9 4 3 7 0 4 6 8 9 9 0]
[9 7 0 2 0 0 9 1 6 4 4 3 4 3 4 6 8 7 9 0]
0.65
Iteration:  50
[0 6 4 1 7 8 1 2 7 6 4 1 7 8 8 4 7 6 1 0]
[0 6 4 1 7 3 1 9 4 6 9 1 2 1 3 1 9 2 3 6]
0.4
Iteration:  60
[4 8 6 0 7 1 6 6 7 7 0 7 1 8 6 9 0 7 7 7]
[9 8 6 3 1 8 6 6 9 9 0 7 1 3 6 3 0 7 1 9]
0.5
Iteration:  70
[8 1 9 6 2 3 8 0 1 0 9 6 3 7 1 4 1 9 9 8]
[2 1 9 2 8 4 5 0 1 0 2 6 3 7 1 2 8 3 4 8]
0.5
Iteration:  80
[0 1 6 4 0 4 4 8 2 8 4 1 8 8 7 8 7 6 8 7]
[6 5 6 9 5 4 4 8 2 4 4 1 8 8 7 3 7 2 8 4]
0.6
Iteration:  90
[6 4 4 0 7 8 8 1 0 4 9 6 1 8 9 2 4 0 2 0]
[6 4 9 5 7 8 

## Test

In [40]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

In [41]:
labels = []
predictions = []

for elem in tfds.as_numpy(ds_val):
    X, Y = preprocess(elem)
    labels.extend(Y)
    predictions.extend(make_predictions(X, W1, b1, W2, b2))
    
print('accuracy:', sum([x == y for x,y in zip(labels, predictions)])/len(labels))

accuracy: 0.8951
